<a href="https://colab.research.google.com/github/probml/probml-notebooks/blob/main/notebooks/gplvm_mocap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Gaussian proceess latent  variable model for motion capture data

http://inverseprobability.com/gpy-gallery/gallery/cmu-mocap-gplvm


## Setup

In [1]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})

In [2]:
# Python Open Data Science
%pip install --upgrade git+https://github.com/lawrennd/ods
import pods


  Cloning https://github.com/lawrennd/ods to /tmp/pip-req-build-4ybunu0n
  Running command git clone -q https://github.com/lawrennd/ods /tmp/pip-req-build-4ybunu0n
  Created wheel for pods: filename=pods-0.1.13-py3-none-any.whl size=59696 sha256=ce0ce8a709925b469715df3e95f6662afb184e56f82dd465a56bc05c48df5bc0
  Stored in directory: /tmp/pip-ephem-wheel-cache-i7ghcwau/wheels/6a/a3/48/b6fc5af7698708ba7256b30afc072c1443b73f1211935dabd2
Successfully built pods


In [3]:
# Machine Learning and Adaptive Intelligence course in Sheffield in 2013.
%pip install --upgrade git+https://github.com/lawrennd/mlai.git
import mlai

  Cloning https://github.com/lawrennd/mlai.git to /tmp/pip-req-build-59npx7jw
  Running command git clone -q https://github.com/lawrennd/mlai.git /tmp/pip-req-build-59npx7jw
  Created wheel for mlai: filename=mlai-0.1.2-py3-none-any.whl size=42919 sha256=111b550379dfe8610d381585d177a83d86b97e186cbc2f9de62d9bd0419c54d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-7k5603v5/wheels/44/a9/67/a02ff08d1d693ec27a8c3204d460a89e9c0d904f15d925b164
Successfully built mlai


In [4]:
%pip install --upgrade git+https://github.com/SheffieldML/GPy.git

  Cloning https://github.com/SheffieldML/GPy.git to /tmp/pip-req-build-02s73or6
  Running command git clone -q https://github.com/SheffieldML/GPy.git /tmp/pip-req-build-02s73or6
     |████████████████████████████████| 71 kB 5.1 MB/s 
  Created wheel for GPy: filename=GPy-1.10.0-cp37-cp37m-linux_x86_64.whl size=3432841 sha256=9ef3e7e0d3880ca88dd10d43a3d1958d623c27ad165315da46d360527e9fb6f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-zm0rcagj/wheels/70/17/ed/27882a196a9af81abc1374f0ae9276c6f018e069aba933419a
  Created wheel for paramz: filename=paramz-0.9.5-py3-none-any.whl size=102566 sha256=c3b4f541670971979bc62bfe7a9edfb44d33d2262202991c8215e8ccfe5e3beb
  Stored in directory: /root/.cache/pip/wheels/c8/95/f5/ce28482da28162e6028c4b3a32c41d147395825b3cd62bc810
Successfully built GPy paramz


## CMU Mocap Database



Motion capture data from the CMU motion capture data base (CMU Motion
Capture Lab, 2003).

You can download any subject and motion from the data set. Here we will
download motion `01` from subject `35`.

In [5]:
subject='35' 
motion=['01']

In [6]:
data = pods.datasets.cmu_mocap(subject, motion)

Acquiring resource: cmu_mocap

Details of data: 
CMU Motion Capture data base. Captured by a Vicon motion capture system consisting of 12 infrared MX-40 cameras, each of which is capable of recording at 120 Hz with images of 4 megapixel resolution. Motions are captured in a working volume of approximately 3m x 8m. The capture subject wears 41 markers and a stylish black garment.

Please cite:
Please include this in your acknowledgements: The data used in this project was obtained from mocap.cs.cmu.edu.\nThe database was created with funding from NSF EIA-0196217.

Data will be stored in /root/ods_data_cache/cmu_mocap.

You must also agree to the following license:
From http://mocap.cs.cmu.edu. This data is free for use in research projects. You may include this data in commercially-sold products, but you may not resell this data directly, even in converted form. If you publish results obtained using this data, we would appreciate it if you would send the citation to your published paper

The data dictionary contains the keys ‘Y’ and ‘skel,’ which represent
the data and the skeleton..

In [7]:
data['Y'].shape

(90, 62)

The data was used in the hierarchical GP-LVM paper (Lawrence and Moore,
2007) in an experiment that was also recreated in the Deep Gaussian
process paper (Damianou and Lawrence, 2013).

In [8]:
print(data['citation'])

Please include this in your acknowledgements: The data used in this project was obtained from mocap.cs.cmu.edu.\nThe database was created with funding from NSF EIA-0196217.


And extra information about the data is included, as standard, under the
keys `info` and `details`.

In [9]:
print(data['info'])
print()
print(data['details'])

Subject: 35. Training motions: 01. Data is sub-sampled to every 4 frames.

CMU Motion Capture data base. Captured by a Vicon motion capture system consisting of 12 infrared MX-40 cameras, each of which is capable of recording at 120 Hz with images of 4 megapixel resolution. Motions are captured in a working volume of approximately 3m x 8m. The capture subject wears 41 markers and a stylish black garment.


## Fit GP-LVM



The original data has the figure moving across the floor during the
motion capture sequence. We can make the figure walk ‘in place,’ by
setting the x, y, z positions of the root node to zero. This makes it
easier to visualize the result.

In [10]:
# Make figure move in place.
data['Y'][:, 0:3] = 0.0

We can also remove the mean of the data.

In [11]:
Y = data['Y']

Now we create the GP-LVM model.

In [12]:
import GPy

In [13]:
model = GPy.models.GPLVM(Y, 2, normalizer=True)

 /usr/local/lib/python3.7/dist-packages/GPy/util/normalizer.py:94: UserWarning:Some values of Y have standard deviation of zero. Resetting to 1.0 to avoid divide by zero errors.


Now we optimize the model.

In [14]:
model.optimize(messages=True, max_f_eval=10000)

In [15]:
import matplotlib.pyplot as plt
import mlai.plot as plot
import mlai.mlai as ma

In [16]:
%matplotlib notebook

In [17]:
import numpy as np

In [20]:
fig, _ = plt.subplots(figsize=plot.big_wide_figsize)
latent_axes = fig.add_subplot(131)
sense_axes = fig.add_subplot(132)
viz_axes = fig.add_subplot(133, projection='3d')

model.plot_latent(ax=latent_axes)
latent_axes.set_aspect('equal')

y = model.Y[0, :]

data_show = GPy.plotting.matplot_dep.visualize.skeleton_show(y[None, :], data['skel'], viz_axes)

lvm_visualizer = GPy.plotting.matplot_dep.visualize.lvm(model.X[0].copy(), model, data_show, latent_axes=latent_axes, sense_axes=sense_axes)
ma.write_figure(figure=fig,
                  filename='cmu-mocap-gplvm.svg', 
                  directory = './gplvm')

<IPython.core.display.Javascript object>

In [21]:
lvm_visualizer = GPy.plotting.matplot_dep.visualize.lvm(model.X[0].copy(), model, data_show, latent_axes=latent_axes, sense_axes=sense_axes)
ma.write_figure(figure=fig,
                  filename='cmu-mocap-gplvm.pdf', 
                  directory = './gplvm')

<img src="https://inverseprobability.com/gpy-gallery/assets/diagrams//gplvm/cmu-mocap-gplvm.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>Gaussian process latent variable model visualisation of CMU
motion capture data set.</i>